In [3]:
import pandas as pd

In [4]:
# Load the dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# RowNumber, CustomerId, and Surname are not that important features

In [5]:
# Preprocess the data

# Drop irrelevant columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Apply label encoding for Gender

In [6]:
# Geograph and Gender both are categorical variable
# So we will some kind of Encoding

from sklearn.preprocessing import LabelEncoder
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


### Apply OneHot encoding for Geography

In [7]:
# OneHot Encoding on Geography column
from sklearn.preprocessing import OneHotEncoder
onehot_geography = OneHotEncoder()
geography_encoder = onehot_geography.fit_transform(data[['Geography']])
geography_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
geography_encoder.toarray

<bound method _cs_matrix.toarray of <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>>

In [9]:
onehot_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geography_encoded_df = pd.DataFrame(geography_encoder.toarray(), columns=onehot_geography.get_feature_names_out(['Geography']))
geography_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [11]:
# Combine OneHot encoded columns with the original data
data = pd.concat([data.drop('Geography', axis=1), geography_encoded_df], axis=1)

In [12]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
# Check Null Values
data.isnull().sum()

CreditScore          0
Gender               0
Age                  0
Tenure               0
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
Geography_France     0
Geography_Germany    0
Geography_Spain      0
dtype: int64

In [14]:
# Save the encoders
import pickle

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_geography.pkl', 'wb') as file:
    pickle.dump(onehot_geography, file)

### Independent and Dependent Features

In [15]:
# Divide the dataset into independent and dependent features

# 'Exited' is our dependent features and remaining all are our independent features
X = data.drop('Exited', axis=1)
y = data['Exited']

### Train and Test Split

In [16]:
# Train and test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Scale these features

In [17]:
# Scale these features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
# Save this scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

----------------------------------------------------------------------------

## ANN Implementation

### Key parameters we will be specifically using:
1. When we start creating our ANN, we have to initialize a "Sequential Network".
2. Whenever we need to create a hidden neuron (not hidden layer), we will be specifically using "Dense".
3. We know that in every node we apply an "Activation Function", it can be sigmoid, tanh, ReLU, Leaky ReLU, etc.
4. In Output node, we use only Sigmoid Activation Function (for binary classification) and Softmax Activation Function (for multiclass classification).
5. Optimizer: It is useful in back propagation because this are responsiblle for updating the weight.
6. Loss Function
7. Metrics: accuracy, mae, mse.
8. Training info: Store the logs in some folder, so that we can use TensorBoard (the main aim of this library is to display this logs in a way that we will be able to understand).


In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [ ]:
(X_train.shape[1],)     # It means it has a single dimension and it has 12 inputs

12

In [21]:
# Build our ANN Model
model = Sequential([

    # HL1 connected with input layer
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),

    # HL2
    Dense(32, activation='relu'),

    # Output layer (Binary Classification)
    Dense(1, activation='sigmoid')
])

e:\Projects\ANN Classification\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# Parameters is combination of weights and bias

In [24]:
# In order to do forward and backward propagration we need to compile this model

In [26]:
opt = tf.keras.optimizers.Adam(learning_rate = 0.01)

In [27]:
# Compile the model
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])   # Metrics parameter is given in the form of list because we can add multiple parameters

In [29]:
# Set up the tensorboard
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Whenever we train our model, it will store all the logs here

tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [30]:
# Set up Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [31]:
# Train the model
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8338 - loss: 0.4037 - val_accuracy: 0.8560 - val_loss: 0.3593
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8524 - loss: 0.3589 - val_accuracy: 0.8545 - val_loss: 0.3438
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8568 - loss: 0.3509 - val_accuracy: 0.8630 - val_loss: 0.3384
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8604 - loss: 0.3451 - val_accuracy: 0.8610 - val_loss: 0.3433
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8558 - loss: 0.3425 - val_accuracy: 0.8555 - val_loss: 0.3562
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8593 - loss: 0.3374 - val_accuracy: 0.8600 - val_loss: 0.3440
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8608 - loss: 0.3376 - val_accuracy: 0.8595 - val_loss: 0.3381
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8646 - loss: 0.3332 - val_accu

In [32]:
model.save('model.h5')

In [33]:
# Load tensorboard extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit